# Bridge Crunchbase - AIDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

from src.ingestion import download_from_drive, import_palette_json

In [2]:
# import databases
crunchbase_5k = download_from_drive("estrazione_crunchbase_5k")
crunchbase = crunchbase_5k

aida_s_anagrafica = download_from_drive("csv_startup_anagrafica")


In [3]:
crunchbase[['Organization Name', 'Postal Code']]

,Organization Name,Postal Code
0,Enilive,144
1,Electrolux Professional,NaN
2,NanoPhoria,20122
3,Akamas,20158
4,Arsenale Bioyards,20122
...,...,...
4995,Gustochef,33028
4996,Fulgur Lab,NaN
4997,Ivy Farm,4022
4998,Local Green,27050


In [4]:
aida_s_anagrafica.head()

,Unnamed: 0,Ragione sociale,BvD ID number,Codice fiscale,Numero CCIAA,Partita IVA,Indirizzo sede legale,Provincia,Indirizzo sede legale - Codice postale,Indirizzo sede legale - Regione,Sede operativa,Sede operativa - Provincia,Sede operativa - Codice postale,Sede operativa - Regione - Regione,ATECO 2007\ncodice,Anno di costituzione,Fondazione,Stato giuridico,Forma giuridica
0,1.0,RAINSHOW S.R.L.,IT03595550835,3595550835,ME0248323,3.595551e+09,"VIA FRANCESCO CRISPI, 4",Messina,98066.0,Sicilia,NaN,NaN,NaN,NaN,711210.0,06/12/2019,NaN,Attiva,S.R.L.
1,2.0,CHECKSIG S.R.L. SOCIETA' BENEFIT,IT11028330964,11028330964,MI2573724,1.102833e+10,"PZA DEL LIBERTY, 8",Milano,20121.0,Lombardia,PIAZZA DEL LIBERTY 8,Milano,20121.0,LOMBARDIA,620100.0,25/10/2019,NaN,Attiva,S.R.L.
2,3.0,OTOFARMA S.P.A.,IT08431301210,8431301210,NA0957454,8.431301e+09,"VLE ANTONIO GRAMSCI, 17/B",Napoli,80122.0,Campania,VIA VIC S.M. DEL PIANTO TORRE 2,Napoli,80144.0,CAMPANIA,266002.0,13/07/2016,NaN,Attiva,S.P.A.
3,4.0,SIKELIA OIL S.R.L.,IT03577640836,3577640836,ME0247028,3.577641e+09,"VIA STRETTO I COCCOMELLI, 64",Messina,98051.0,Sicilia,NaN,NaN,NaN,NaN,473000.0,15/07/2019,NaN,Attiva,S.R.L.
4,5.0,POWER.IT S.R.L.,IT04355290166,4355290166,BG0456683,4.355290e+09,"VIA ALESSANDRO NOLI, 13",Bergamo,24125.0,Lombardia,NaN,NaN,NaN,NaN,352300.0,31/01/2019,NaN,Attiva,S.R.L.


In [5]:
# AIDA

# Settare index su colonna Unamed: 0
aida_s_anagrafica.set_index("Unnamed: 0", inplace=True)

# Rinominare colonna Unnamed: 0 in Index
aida_s_anagrafica.index.name = "Index"

# Rinominare colonna ATECO 2007 codice in ATECO
aida_s_anagrafica.rename(columns={"ATECO 2007\ncodice": "ATECO"}, inplace=True)

## Data Cleaning

# Remove rows where ATECO is empty or NaN
aida_s_anagrafica.dropna(subset=['ATECO'], inplace=True)

# Remove column Fondazione (beacuse is empty)
aida_s_anagrafica.drop(columns=['Fondazione'], inplace=True)

# Convert ATECO in integer
aida_s_anagrafica['ATECO'] = aida_s_anagrafica['ATECO'].astype(int)

# Convert the column where in the title there is "Codice Postale" in integer
aida_s_anagrafica.loc[:, aida_s_anagrafica.columns.str.contains('Codice postale')] = aida_s_anagrafica.loc[:, aida_s_anagrafica.columns.str.contains('Codice postale')].astype('Int64')

# Convert the index column to integer
aida_s_anagrafica.index = aida_s_anagrafica.index.astype(int)

# Convert column Partita IVA to see entirely
aida_s_anagrafica["Partita IVA"] = aida_s_anagrafica["Partita IVA"].apply(lambda x: '{:.0f}'.format(x))
aida_s_anagrafica["Partita IVA"]

# if Sede operativa - Provincia is NaN, insert the value contained in Provincia
aida_s_anagrafica['Sede operativa - Provincia'].fillna(aida_s_anagrafica['Provincia'], inplace=True)

# if Sede operativa - Codice postale is NaN or <NA>, insert the value contained in Indirizzo sede legale - Codice postale
aida_s_anagrafica['Sede operativa - Codice postale'].fillna(aida_s_anagrafica['Indirizzo sede legale - Codice postale'], inplace=True)

# if Sede operativa - Regione - Regione is NaN, insert the value contained in Indirizzo sede legale - Regione
aida_s_anagrafica['Sede operativa - Regione - Regione'].fillna(aida_s_anagrafica['Indirizzo sede legale - Regione'], inplace=True)

# Strip any leading or trailing spaces from the column names
aida_s_anagrafica.columns = aida_s_anagrafica.columns.str.strip()

# Filter the dataframe
aida_s_anagrafica[aida_s_anagrafica["Anno di costituzione"].str[-4:].astype(int) > 2017][["Ragione sociale", "Indirizzo sede legale - Codice postale"]]

/tmp/ipykernel_30839/3014909584.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[98066, 20121, 80122, 98051, 24125, 90135, 80143,  6012,  8011, 72017,
 ...
 15076, 15121, 15121, 92013, 92019, 92026, 92100,  <NA>, 92100, 92100]
Length: 11954, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  aida_s_anagrafica.loc[:, aida_s_anagrafica.columns.str.contains('Codice postale')] = aida_s_anagrafica.loc[:, aida_s_anagrafica.columns.str.contains('Codice postale')].astype('Int64')
/tmp/ipykernel_30839/3014909584.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ <NA>, 20121, 80144,  <NA>,  <NA>, 90135, 80100,  <NA>,  8100, 72017,
 ...
  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>]
Length: 11954, dtype: Int64' has dtype incompatible with f

,Ragione sociale,Indirizzo sede legale - Codice postale
Index,,
1,RAINSHOW S.R.L.,98066
2,CHECKSIG S.R.L. SOCIETA' BENEFIT,20121
4,SIKELIA OIL S.R.L.,98051
5,POWER.IT S.R.L.,24125
7,MILK UP S.R.L. START UP COSTITUITA A NORMA DEL...,80143
...,...,...
11963,CIFRANDA S.R.L.,92026
11964,SNEARK S.R.L.,92100
11965,MATT LAMPEDUSA S.R.L.,<NA>
